# Automated Feature Selection

In this live lecture activity, we are going to consider the problem of how to write algorithms that automatically make reasonable choices about which features to include in machine learning models. There are many approaches to this problem, but here are two. 

## Grab and Prepare the Titanic Data

In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

In [3]:
url = "https://philchodrow.github.io/PIC16A/datasets/titanic.csv"
        
titanic = pd.read_csv(url)

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
titanic["Sex"] = le.fit_transform(titanic['Sex'])
titanic = titanic.drop(["Name"], axis = 1)

X = titanic.drop(['Survived'], axis = 1)
y = titanic['Survived']

In [ ]:
y

## Exhaustive Search

The problem of enumerating combinations of objects gets very difficult very quickly: for example, in a data set with 10 predictor columns, there are $2^{10} = 1024$  possible subsets of these columns. Worse, many data sets have many more columns than this! But, for relatively small data sets, it can sometimes be practical to simply try all possible combinations of columns, assessing them using cross-validation in order to settle on a good solution. 

The `itertools` module is exceptionally useful for this. 

In [30]:
from itertools import combinations

captains = ["Picard", "Sisko", "Janeway", "Georgiou"]

# all pairs
list(combinations(captains,2))

# all triples

list(combinations(captains,3))

[('Picard', 'Sisko', 'Janeway'),
 ('Picard', 'Sisko', 'Georgiou'),
 ('Picard', 'Janeway', 'Georgiou'),
 ('Sisko', 'Janeway', 'Georgiou')]

In [42]:
def exhaustive_search(model, X, y, min_cols, max_cols):
    best_cv = 0
    best_cols = None
    for n_cols in range(min_cols, max_cols + 1):
        for cols in combinations(X.columns, n_cols):
            cv = cross_val_score(model, X[list(cols)], y, cv = 10).mean()
            if cv > best_cv:
                best_cv = cv
                best_cols = cols
    
    return best_cv, best_cols

In [49]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()

best_cv, best_cols = exhaustive_search(LR, X, y, 2, 5)

In [50]:
best_cv, best_cols

(0.8016215526046986,
 ('Pclass', 'Sex', 'Age', 'Siblings/Spouses Aboard', 'Fare'))

## Greedy Stagewise Feature Selection

Here's what we are going to do. We will start with one randomly-chosen "active" column. Then, we will do the following a user-specified number of times: 

1. Compute the CV score of a model using only the active columns, and save it. 
2. Propose either "activating" or "deactivating" a column (i.e. adding or removing it from the list of active columns). Compute the CV score. If the CV score has improved, accept the proposal (i.e. add that column to the active set, or remove it).

## Part A: Setup

In [9]:
# import logistic regression and cross-validation

from sklearn.model_selection import cross_val_score

# create a logistic regression model
LR = LogisticRegression(solver = "liblinear")

In [10]:
def initialize_lists():
    """
    Create an "active" list with a single random column
    from X.columns and an "inactive" list with 
    all remaining columns. 
    """
    # grab a single random column
    active = [np.random.choice(X.columns)]
    
    # make a list of all the other columns
    inactive = list(X.columns)
    inactive.remove(active[0])
    return active, inactive
    
def move(col, active, inactive, mode = "activate"):
    """
    Activate or deactivate a single column
    by moving it between the active and inactive
    lists. 
    Does not modify active or inactive -- instead 
    returns copies. 
    """
    # create copies
    new_active = active.copy()
    new_inactive = inactive.copy()
    
    if mode == "activate":
        # if we are activating a column
        new_inactive.remove(col)
        # add col to the active list
        new_active.append(col)
    
    # if we are deactivating a column
    
    if mode == "deactivate":
        new_active.remove(col)
        new_inactive.append(col)
    
    # return copies
    return new_active, new_inactive

### Illustrations

In [11]:
active, inactive = initialize_lists()
active, inactive

(['Fare'],
 ['Pclass',
  'Sex',
  'Age',
  'Siblings/Spouses Aboard',
  'Parents/Children Aboard'])

In [12]:
move("Age", active, inactive, mode = "activate")

(['Fare', 'Age'],
 ['Pclass', 'Sex', 'Siblings/Spouses Aboard', 'Parents/Children Aboard'])

## Part B: Greedy Stagewise Selection

In [13]:
def greedy_stagewise_feature_selection(model, X, y, n_iters = 20):
    
    # initialize with a single, randomly selected column
    active, inactive = initialize_lists()
    
    # initialize the best CV score
    best_CV = 0
    
    # main loop, n_iters times
    for i in range(n_iters):
        # alternate between activating and deactivating
        for mode in ["activate", "deactivate"]:
        
            # if mode is "activate" and there are any remaining inactive
            # columns, randomly select one. Otherwise, continue
            if (mode == "activate"):
                if len(inactive) > 0:
                    col = np.random.choice(inactive)
                else: 
                    continue
            
            # if mode is "deactivate" and if there at least 2 active
            # columns then pick a random active column
            if (mode == "deactivate") and (len(active) >= 2):
                col = np.random.choice(active)
            
            # create a new, proposed active list by moving
            # col between lists
            
            new_active, new_inactive = move(col, active, inactive, mode)
            
            # compute the CV score
            CV_score = cross_val_score(LR, X[new_active], y, cv = 10).mean()
            
            # if the CV score is an improvement, update the 
            # active and inactive column sets. 
            
            if (CV_score > best_CV) and (len(new_active) >=1):
                best_CV = CV_score
                active = new_active
                inactive = new_inactive
            
            print("Number of columns: " + str(len(active)) + ". CV score: " + str(best_CV))
    return active

In [21]:
best_cols = greedy_stagewise_feature_selection(LR, X, y, n_iters = 10)

Number of columns: 2. CV score: 0.7812862677879551
Number of columns: 2. CV score: 0.7812862677879551
Number of columns: 3. CV score: 0.7880532790544039
Number of columns: 3. CV score: 0.7880532790544039
Number of columns: 3. CV score: 0.7880532790544039
Number of columns: 3. CV score: 0.7880532790544039
Number of columns: 4. CV score: 0.7891870659024764
Number of columns: 4. CV score: 0.7891870659024764
Number of columns: 4. CV score: 0.7891870659024764
Number of columns: 4. CV score: 0.7891870659024764
Number of columns: 5. CV score: 0.7948202903208527
Number of columns: 5. CV score: 0.7948202903208527
Number of columns: 5. CV score: 0.7948202903208527
Number of columns: 5. CV score: 0.7948202903208527
Number of columns: 5. CV score: 0.7948202903208527
Number of columns: 4. CV score: 0.7981680861320906
Number of columns: 4. CV score: 0.7981680861320906
Number of columns: 3. CV score: 0.8004178049172425
Number of columns: 3. CV score: 0.8004178049172425
Number of columns: 3. CV score:

In [22]:
cols

['Sex', 'Siblings/Spouses Aboard', 'Pclass']